In [ ]:
# This code was inspired in the text generator of http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
# We have added nltk toos for working with words instead of letters
# the advantage is that is that we do not need to modelate the vocabulary,
# instead we need to modelate only the syntaxis
# the disadvantage is that we moove from 48 letters to 2000+ words in our vocabulary


In [4]:
import nltk
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [5]:
""" This part of the code is if we want to limit the vocabulary
from collections import Counter
def most_common(self, n=None):
    '''List the n most common elements and their counts from the most
    common to the least.  If n is None, then list all element counts.

    >>> Counter('abcdeabcdabcaba').most_common(3)
    [('a', 5), ('b', 4), ('c', 3)]

    '''
    # Emulate Bag.sortedByCount from Smalltalk
    if n is None:
        return sorted(self.iteritems(), key=_itemgetter(1), reverse=True)
    return _heapq.nlargest(n, self.iteritems(), key=_itemgetter(1))
print(Counter([1,1,1,3,3,3,4]).most_common(1) )
"""


"from collections import Counter\ndef most_common(self, n=None):\n    '''List the n most common elements and their counts from the most\n    common to the least.  If n is None, then list all element counts.\n\n    >>> Counter('abcdeabcdabcaba').most_common(3)\n    [('a', 5), ('b', 4), ('c', 3)]\n\n    '''\n    # Emulate Bag.sortedByCount from Smalltalk\n    if n is None:\n        return sorted(self.iteritems(), key=_itemgetter(1), reverse=True)\n    return _heapq.nlargest(n, self.iteritems(), key=_itemgetter(1))\nprint(Counter([1,1,1,3,3,3,4]).most_common(1) )\n"

In [7]:

# We start by creating the dictionaries between the strings and the one-hot encodings


import unidecode


with open("wonderland.txt") as f:
    raw_text = f.read().lower()
    
    # decode the string
    #to_tokenize = raw_text.decode('utf-8').encode("ascii","replace")#, "ignore")
    to_tokenize = unidecode.unidecode(raw_text.decode("utf-8"))
    
    # tokenize the string
    tokens = nltk.word_tokenize(to_tokenize)

    # create the one-hot dictionaries
    word_to_int = dict((c, i) for i, c in enumerate(set(tokens)))
    int_to_word = dict((i, c) for i, c in enumerate(set(tokens)))
    print(len(word_to_int.keys() ))
    n_vocab = 8000

    # The folowing is a option that we will explore later
    """one_hot_st = one_hot(to_tokenize,
                                 n_vocab,
                                 filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                 lower=True,
                                 split=" ")"""
    

## We create our features X and target Y    
    
n_words = len(tokens)    
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
    seq_in = tokens[i:i + seq_length]
    seq_out = tokens[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print([int_to_word[number] for number in dataX[1]])
#print(dataY[1])


2874
['ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', 'you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org', 'title', ':', 'alice', "'s", 'adventures', 'in', 'wonderland', 'author', ':', 'lewis', 'carroll', 'posting', 'date', ':', 'june', '25', ',', '2008', '[', 'ebook', '#', '11', ']', 'release', 'date', ':', 'march', ',', '1994', 'last', 'updated', ':', 'october', '6', ',', '2016', 'language', ':', 'english', 'character', 'set', 'encoding', ':', 'utf-8', '***', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'alice', "'s", 'adventures', 'in']


In [8]:
"""# replace the less frecuent words (in the case of having huge vocabularies)
fdist = nltk.FreqDist(word for word in tokens)
vocab = fdist.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
print(len(word_to_index.keys() ))
print(word_to_index )"""

'# replace the less frecuent words (in the case of having huge vocabularies)\nfdist = nltk.FreqDist(word for word in tokens)\nvocab = fdist.most_common(vocabulary_size-1)\nindex_to_word = [x[0] for x in vocab]\nindex_to_word.append(unknown_token)\nword_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])\nprint(len(word_to_index.keys() ))\nprint(word_to_index )'

In [9]:
import numpy
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
#X = numpy.reshape(dataX, (150836, 100, 1)) 
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [10]:
# Later we define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]))) # 256 corresponds to the number of neurons of each lstm cell
#include a dropouto for regularization
model.add(Dropout(0.2))
# Last layer (dense)
model.add(Dense(y.shape[1], activation='softmax'))
# compile with categorical_crossentropy loss and adam optimizer. We add metrics = accuracy for monitoring
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])




In [29]:
# define the checkpoint
filepath="earlystopping-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

from keras.callbacks import EarlyStopping
#we add early stoping if the models does not improove in patience steps
early_stopping_monitor = EarlyStopping(patience = 2) #alow only 2 iterations with no improvement
# create a callbacklist
callbacks_list = [checkpoint,early_stopping_monitor]
# fit the model for 50 epochs


In [30]:

model.fit(X, y, epochs=50, batch_size=128, callbacks= callbacks_list, validation_split = 0.2)


Train on 26651 samples, validate on 6663 samples
Epoch 1/50
26651/26651 [==============================] - 440s - loss: 5.8337 - acc: 0.0719 - val_loss: 6.0932 - val_acc: 0.0752
Epoch 2/50
26651/26651 [==============================] - 410s - loss: 5.8168 - acc: 0.0719 - val_loss: 6.1108 - val_acc: 0.0752
Epoch 3/50
26651/26651 [==============================] - 409s - loss: 5.8115 - acc: 0.0719 - val_loss: 6.1364 - val_acc: 0.0752
Epoch 4/50
26651/26651 [==============================] - 409s - loss: 5.8093 - acc: 0.0720 - val_loss: 6.1395 - val_acc: 0.0752


In [31]:


# load the network weights
filename = "earlystopping-weights-improvement-03-5.8093.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')




In [73]:
#int_to_word = dict((i, c) for i, c in enumerate(set(tokens)))
#print(int_to_word)


In [33]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print(pattern)
print "Seed:"
print "\"", ''.join([int_to_word[value] + " " for value in pattern]), "\""
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."



[187, 1417, 729, 1311, 2300, 512, 321, 1519, 1886, 187, 2443, 411, 589, 1343, 2128, 1332, 2483, 1860, 1643, 2443, 1098, 2577, 1020, 550, 1608, 1098, 217, 512, 465, 550, 1608, 2260, 596, 975, 1668, 1541, 2631, 187, 2517, 673, 1567, 1391, 79, 2302, 204, 140, 1519, 550, 2846, 2128, 550, 1743, 673, 1527, 1388, 1668, 396, 1563, 512, 550, 1285, 187, 903, 1668, 2323, 2253, 550, 785, 673, 1988, 176, 550, 1971, 187, 2831, 1668, 1541, 654, 2517, 2202, 1519, 2091, 1567, 1391, 892, 465, 2649, 729, 766, 288, 468, 2768, 512, 1084, 892, 1643, 2443, 589, 411, 1519]
Seed:
" , there 's no use in talking to him , ' said alice desperately : 'he's perfectly idiotic ! ' and she opened the door and went in . the door led right into a large kitchen , which was full of smoke from one end to the other : the duchess was sitting on a three-legged stool in the middle , nursing a baby ; the cook was leaning over the fire , stirring a large cauldron which seemed to be full of soup . 'there 's certainly too much pepp